# Cyber Security Salary Analysis
More info: https://www.kaggle.com/datasets/deepcontractor/cyber-security-salaries

<img src="https://storage.googleapis.com/kaggle-datasets-images/2392731/4039045/59320bd49b4c7491a6f3223e8e280cdc/dataset-cover.PNG?t=2022-08-07-07-52-01" style="width: 300px;">

In [1]:
# importing dependecies
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from pandas_profiling import ProfileReport
import numpy as np

/root/anaconda3/envs/exp1/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



In [2]:
# settings
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
np.random.seed(1000)
from plotly import offline
offline.init_notebook_mode(connected=True)

In [3]:
df = pd.read_csv('salaries_cyber.csv')
df

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2022,EN,FT,Cyber Program Manager,63000,USD,63000,US,50,US,S
1,2022,MI,FT,Security Analyst,95000,USD,95000,US,0,US,M
2,2022,MI,FT,Security Analyst,70000,USD,70000,US,0,US,M
3,2022,MI,FT,IT Security Analyst,250000,BRL,48853,BR,50,BR,L
4,2022,EN,CT,Cyber Security Analyst,120000,USD,120000,BW,100,BW,S
...,...,...,...,...,...,...,...,...,...,...,...
1242,2020,MI,FT,Cyber Security Analyst,140000,AUD,96422,AU,50,AU,M
1243,2021,SE,FT,Information Security Manager,60000,GBP,82528,GB,50,GB,L
1244,2021,SE,FT,Penetration Testing Engineer,126000,USD,126000,US,100,US,L
1245,2021,MI,FT,Information Security Analyst,42000,GBP,57769,GB,100,GB,L


# Exploring the DataFrame

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1247 entries, 0 to 1246
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           1247 non-null   int64 
 1   experience_level    1247 non-null   object
 2   employment_type     1247 non-null   object
 3   job_title           1247 non-null   object
 4   salary              1247 non-null   int64 
 5   salary_currency     1247 non-null   object
 6   salary_in_usd       1247 non-null   int64 
 7   employee_residence  1247 non-null   object
 8   remote_ratio        1247 non-null   int64 
 9   company_location    1247 non-null   object
 10  company_size        1247 non-null   object
dtypes: int64(4), object(7)
memory usage: 107.3+ KB


In [5]:
df.describe()

,work_year,salary,salary_in_usd,remote_ratio
count,1247.000000,1.247000e+03,1247.000000,1247.000000
mean,2021.316760,5.608525e+05,120278.218925,71.491580
std,0.715501,1.415944e+07,70291.394942,39.346851
min,2020.000000,1.740000e+03,2000.000000,0.000000
25%,2021.000000,7.975450e+04,74594.500000,50.000000
50%,2021.000000,1.200000e+05,110000.000000,100.000000
75%,2022.000000,1.600800e+05,150000.000000,100.000000
max,2022.000000,5.000000e+08,910991.000000,100.000000


In [29]:
# creating profile
ProfileReport(
    df,
    title="Cyber Security Salaries Profile"   
)

Render HTML: 100%|██████████| 1/1 [00:01<00:00,  1.20s/it]


#### <pre>    There are some duplicated rows, but since there is no id or other feature to make sure they are the same person, i will give the benefit of the doubt. It could be that co-workers with the same position answered the survey.
#### <pre>    There is some correlation between the features which we shall explore further, but it makes me believe that a model could be made to help predict <strong>salary in usd</strong>.

# Investigating and Visualizing the Dataset

In [30]:
px.histogram(df, x='salary_in_usd')

In [31]:
# plotting boxplots to see the relation of salary and other features
fig = px.box(df, x='experience_level', y='salary_in_usd', points='all', width=1000, height=400,
             category_orders={'experience_level':['EN', 'MI', 'SE', 'EX']}, color='experience_level')
fig.show()
fig = px.box(df, x='employment_type', y='salary_in_usd', points='all', width=1000, height=400,
             category_orders={'employment_type':['PT', 'FT', 'CT', 'FL']}, color='employment_type')
fig.show()
fig = px.box(df, x='company_size', y='salary_in_usd', points='all', width=1000, height=400,
             color='company_size')
fig.show()

#### <pre>    Here are some takeaways i got from these plots. First, the salary distribution is left skewed, similar to many other income distributions, and the highest bin include professionals who earn $100k-$110k. Higher  <strong>experience level</strong> indeed usually means higher <strong>salary in usd</strong> and executives are the ones who earn the most. Full time jobs have the highest salaries, followed by contract jobs, but unfortunately there were not a lot of freelancers to get a better understanding of their condition. Interesting enough, medium sized companies have higher median salaries compared to large ones, but in large companies it is unlikely, but possible to earn really high salaries.

#### <pre>    We found out that there are features with different representation, so lets build some histograms to visualize them better

In [32]:
# plotting histograms for different categories
fig = make_subplots(rows=1, cols=3)
fig.add_trace(
    go.Histogram(x=df['experience_level'], texttemplate="%{y}", name='Experience Level'), row=1, col=1
)
fig.add_trace(
    go.Histogram(x=df['employment_type'], texttemplate="%{y}", name='Employment Type'), row=1, col=2
)
fig.add_trace(
    go.Histogram(x=df['company_size'], texttemplate="%{y}", name='Company Size'), row=1, col=3
)

#### <pre>    Seeing these histograms i lean to believe cybersecurity is a mature field. Most of the professionals are seniors, full time and work for large companies.

#### <pre>    There is still other factors to consider, like <strong>job title</strong>, <strong>employee residence</strong> and <strong>company location</strong>. As seen in the profile report, those have high cardinality, which means that plotting all the categories would be hard to interpret, but i will show only the 20 most frequent.

In [33]:
top_job_titles = df['job_title'].value_counts()[:20]
top_empl_residence = df['employee_residence'].value_counts()[:20]
top_comp_location = df['company_location'].value_counts()[:20]

fig = px.bar(x=top_job_titles.index, y=top_job_titles, title='Job Title', width=1000, height=600)
fig.show()
fig = px.bar(x=top_empl_residence.index, y=top_empl_residence, title='Employee Residence', width=1000, height=600)
fig.show()
fig = px.bar(x=top_comp_location.index, y=top_comp_location, title='Company Location', width=1000, height=600)
fig.show()

#### <pre>    The lack of representation from places other than US probably means that analysing using <strong>employee residence </strong> or <strong>company location</strong> won't be very accurate, but i will still do it.

In [34]:
# box ploting the salaries of the 20 most frequent job titles
top_job_titles_df = df[df['job_title'].isin(top_job_titles.index)]

fig = px.box(top_job_titles_df, x='job_title', y='salary_in_usd', width=1000, height=400,
             category_orders={'employment_type':top_job_titles.index}, color='job_title')
fig.show()

In [35]:
# plotting median salary on a map
import country_converter as cc
df['company_location'] = cc.convert(names=df['company_location'], to="ISO3")

grouped = df[['company_location', 'salary_in_usd']].groupby('company_location').median().reset_index()

fig = px.choropleth(locations=grouped['company_location'],
                    color=grouped['salary_in_usd'],
                    title = 'Median Salary of Cyber Security Professionals Around the World',
                    width=800,
                    height=600,
                    color_continuous_scale=px.colors.sequential.Oranges)
fig.show()


In [36]:
px.sunburst(top_job_titles_df, path=['job_title', 'experience_level'], width=700, height=700)

#### <pre>    This plot is interesting. As we have already seem, most of the jobs in the cyber security field belong to senior professionals, and indeed <strong>security engineers</strong> are mostly seniors, but <strong>Cyber Security Analysts</strong> are mostly entry and mid level. <strong>Chief Information Security Officers</strong> are mostly executives, which makes sense. There is still more fascinating information in this graph.

#### <pre>    Lets try to plot the same graph, but using <strong>company size</strong> now.

In [37]:
px.sunburst(top_job_titles_df, path=['job_title', 'company_size'], width=700, height=700)

#### <pre>    Indeed, most professionals are working in large companies, but the most common <strong>job title</strong>, <strong>Security Engineer</strong> is mostly associated with medium sized companies. Same for <strong>DevSecOps Engineer</strong>.

#### <pre>    There are still 2 columns left to investigate, <strong>work year</strong> and <strong>remote ratio</strong> and we will do it by plotting both together to see what happened in the last 3 years(since the start of covid) with remote work.

In [38]:
fig = px.histogram(df.astype({'work_year':str}), x='work_year', color='remote_ratio', 
                   category_orders={'work_year':['2020', '2021', '2022'], 'remote_ratio':[0,50,100]}, 
                   width=1000, height=500)
fig.show()

#### <pre>    Remote work increased.

# Building a Model

#### <pre>    With the takeaways we got now, i will try to build some prediction models for <strong>salary in usd</strong>. We've seen that  <strong>experience level</strong>, <strong>employment type</strong>, <strong>job title</strong>, <strong>company size</strong> and <strong>company location</strong> impact <strong>salary in usd</strong> and i believe these features have some independence between each other. A professional with more experience usually earns more independent of the other variables. This and the fact they are categorical features makes me want to use <strong>Naive Bayes</strong>. In this case, the model won't try to predict an exact salary (regression), but it will try to say if a professional earns more than a certain amount or not (binary classification).

#### <pre>    One problem we have is the high cardinality of the column <strong>job title</strong>. There are many entries with very small representation and this could be a problem for the model, so, trying to mitigate this issue, i will group together the <strong>job titles</strong> with similar salary. The way i will do it is by collecting the median <strong>salary in usd</strong> of each <strong>job title</strong> and will apply a one dimensional k-means to get clusters that will be used by the model instead of <strong>job title</strong>. Also, i will group the <strong>company location</strong> entries that are different from USA.

In [39]:
# removing outliers
quant = df["salary_in_usd"].quantile(0.97)
df = df[df["salary_in_usd"] < quant]

In [40]:
# factorize company_location and group non-USA entries 
def not_usa(x):
    if x!='USA':
        return 0
    return 1
df['company_location_fact'] = df['company_location'].apply(not_usa)

In [41]:
# split data into train and test
from sklearn.model_selection import train_test_split

df['experience_level_fact'] = pd.factorize(df['experience_level'])[0]
df['employment_type_fact'] = pd.factorize(df['employment_type'])[0]
df['company_size_fact'] = pd.factorize(df['company_size'])[0]

df['sal>x'] = df['salary_in_usd'] >= 100_000

df_train, df_test, _, _ = train_test_split(df, df['sal>x'], test_size=0.2, stratify=df['sal>x'])
df_train

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,company_location_fact,experience_level_fact,employment_type_fact,company_size_fact,sal>x
1111,2020,SE,FT,Cyber Threat Analyst,140000,USD,140000,US,100,USA,L,1,2,0,2,True
268,2022,SE,FT,Security Engineer,184800,USD,184800,US,100,USA,M,1,2,0,1,True
850,2021,EN,FT,Ethical Hacker,119000,USD,119000,US,100,USA,L,1,0,0,2,True
507,2022,SE,FT,Security Analyst,115000,USD,115000,US,100,USA,L,1,2,0,2,True
388,2022,SE,FT,Application Security Engineer,42000,EUR,45468,RO,100,ROU,L,0,2,0,2,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40,2022,MI,FT,Information Security Officer,97900,USD,97900,US,50,USA,L,1,1,0,2,False
412,2022,SE,FT,Cyber Security Engineer,150000,USD,150000,US,100,USA,L,1,2,0,2,True
701,2022,MI,FT,Information Security Specialist,100000,CAD,78451,CA,100,CAN,L,0,1,0,2,False
1013,2020,SE,FT,Information Security Engineer,106000,USD,106000,US,0,USA,L,1,2,0,2,True


In [42]:
# creating df of median_salaries
median_salaries = df_train[['job_title', 'salary_in_usd']].groupby('job_title').median()
median_salaries

,salary_in_usd
job_title,
Application Security Analyst,54129.0
Application Security Engineer,79905.0
Application Security Specialist,85000.0
Azure Security Engineer,60000.0
Chief Information Security Officer,103655.0
...,...
Threat Intelligence Analyst,138000.0
Threat Intelligence Response Analyst,220000.0
Vulnerability Analyst,115000.0


In [43]:
# clustering the medians and adding as a column in median_salaries
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=15)
median_salaries['cluster'] = kmeans.fit_predict(median_salaries)
median_salaries

,salary_in_usd,cluster
job_title,,
Application Security Analyst,54129.0,7
Application Security Engineer,79905.0,8
Application Security Specialist,85000.0,9
Azure Security Engineer,60000.0,1
Chief Information Security Officer,103655.0,11
...,...,...
Threat Intelligence Analyst,138000.0,12
Threat Intelligence Response Analyst,220000.0,4
Vulnerability Analyst,115000.0,6


In [44]:
px.scatter(median_salaries, x='salary_in_usd', color='cluster')

In [45]:
# join median_salaries in df 
df_train = df_train.set_index('job_title').join(median_salaries['cluster']).reset_index()
df_test = df_test.set_index('job_title').join(median_salaries['cluster']).reset_index()
# drop nan values
df_test.dropna(inplace=True)

In [46]:
px.scatter(df_train, x='salary_in_usd',  y='job_title', color='cluster')

### Fitting in Naive Bayes

In [47]:
from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import train_test_split

X_train = df_train[
    ['experience_level_fact', 'employment_type_fact', 'company_size_fact', 'company_location_fact', 'cluster']
]
y_train = df_train['sal>x']

X_test = df_test[
    ['experience_level_fact', 'employment_type_fact', 'company_size_fact', 'company_location_fact', 'cluster']
]
y_test = df_test['sal>x']

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

nb = CategoricalNB()
nb.fit(X_train, y_train)

CategoricalNB()

In [48]:
from sklearn.metrics import accuracy_score

# train accuracy
accuracy_score(y_train, nb.predict(X_train))

0.8100320170757738

In [49]:
# test accuracy
accuracy_score(y_test, nb.predict(X_test))

0.7922077922077922

### Random Forest for Comparison

In [50]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(X_train, y_train)

RandomForestClassifier()

In [52]:
accuracy_score(y_test, rf.predict(X_test))

0.8311688311688312

#### <pre>    Both models have similar accuracy score and, since salary is an attribute that can vary a lot, it went better than i expected.